In [2]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
train_df = pd.read_csv('../preprocessing_data/train.csv')
test_df = pd.read_csv('../preprocessing_data/test.csv')
X = np.array(train_df.drop(['SalePrice'], axis = 1).values)
y = np.array(train_df['SalePrice'].values)
X_test = np.array(test_df.values)

In [4]:
from bayes_opt import BayesianOptimization
import xgboost as xgb
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import get_scorer, mean_squared_error
import time

In [5]:
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

In [6]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size= 0.3, random_state=42)

In [7]:
def xgbr_cl_bo(colsample_bytree, gamma, learning_rate, max_depth, min_child_weight, n_estimators, reg_alpha,
                reg_lambda, subsample, seed):
    params_lasso = {}
    params_lasso['colsample_bytree'] = colsample_bytree
    params_lasso['gamma'] = gamma
    params_lasso['learning_rate'] = learning_rate
    params_lasso['max_depth'] = round(max_depth)
    params_lasso['min_child_weight'] = min_child_weight
    params_lasso['n_estimators'] = round(n_estimators)
    params_lasso['reg_alpha'] = reg_alpha
    params_lasso['reg_lambda'] = reg_lambda
    params_lasso['subsample'] = subsample
    params_lasso['seed'] = round(seed)
    
    xgbr = xgb.XGBRegressor(**params_lasso)
    xgbr.fit(X_train, y_train)

    score = -rmse(y_valid, xgbr.predict(X_valid))
    return score

# Run Bayesian Optimization
start = time.time()
params_xgbr ={
    'colsample_bytree': (0.5, 1), 
    'gamma': (0, 0.1), 
    'learning_rate': (0.001, 0.01), 
    'max_depth': (5, 10), 
    'min_child_weight': (1, 1.5), 
    'n_estimators': (6000, 7500), 
    'reg_alpha': (0, 0.1),
    'reg_lambda': (0, 0.5), 
    'subsample': (0, 0.5), 
    'seed': (40, 50)
}
xgbr_bo = BayesianOptimization(xgbr_cl_bo, params_xgbr, random_state=111)
xgbr_bo.maximize(init_points=20, n_iter=4)
print('It takes %s minutes' % ((time.time() - start)/60))

|   iter    |  target   | colsam... |   gamma   | learni... | max_depth | min_ch... | n_esti... | reg_alpha | reg_la... |   seed    | subsample |
-------------------------------------------------------------------------------------------------------------------------------------------------
| 1         | -0.1216   | 0.8061    | 0.01691   | 0.004925  | 8.846     | 1.148     | 6.224e+03 | 0.002248  | 0.2101    | 42.39     | 0.1688    |
| 2         | -0.1231   | 0.9954    | 0.02377   | 0.001731  | 8.348     | 1.311     | 6.411e+03 | 0.04662   | 0.05918   | 40.74     | 0.4504    |
| 3         | -0.1313   | 0.897     | 0.08406   | 0.008337  | 9.955     | 1.289     | 7.221e+03 | 0.04213   | 0.01372   | 44.54     | 0.05266   |
| 4         | -0.1231   | 0.9086    | 0.06977   | 0.006088  | 6.371     | 1.499     | 6.207e+03 | 0.06154   | 0.2425    | 44.05     | 0.3639    |
| 5         | -0.1216   | 0.6614    | 0.04005   | 0.003871  | 9.736     | 1.459     | 7.221e+03 | 0.003408  | 0.4717    | 49

In [8]:
xgbr = xgb.XGBRegressor(
                colsample_bytree=0.9954,
                gamma=0.02377,
                learning_rate=0.001731,
                max_depth=8,
                min_child_weight=1.311,
                n_estimators=6411,                                                                  
                reg_alpha=0.04662,
                reg_lambda=0.05918,
                subsample=0.4504,
                seed=40,)
xgbr.fit(X_train, y_train)
rmse(y_valid, xgbr.predict(X_valid))

0.1229453376905725

In [9]:
ans = pd.read_csv('../submission/cheat.csv').drop('Id', axis = 1)
ans = np.array(ans.values).reshape(1, -1)[0]
y_pred = xgbr.predict(X_test)
print(f"Error: {rmse(np.log1p(ans), y_pred)}")

Error: 0.12945373199632956
